# Creating Redshift Cluster using the AWS python SDK 
## An example of Infrastructure-as-code

In [1]:
import pandas as pd
import boto3
import json
import redshift_connector

# STEP 0: Make sure you have an AWS secret and access key

- Create a new IAM user in your AWS account
- Give it `AdministratorAccess`, From `Attach existing policies directly` Tab
- Take note of the access key and secret 
- Edit the file `dwh.cfg` in the same folder as this notebook and fill
<font color='red'>
<BR>
[AWS]<BR>
KEY= YOUR_AWS_KEY<BR>
SECRET= YOUR_AWS_SECRET<BR>
<font/>


# Load DWH Params from a file

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DWH_DB,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,dX4D7U4L1z
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


## Create clients for EC2, S3, IAM, and Redshift

In [3]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

## Check out the sample data sources on S3

In [ ]:
sampleDbBucket =  s3.Bucket("udacity-dend")
for obj in sampleDbBucket.objects.filter(Prefix="song-data"):
    print(obj)
# for obj in sampleDbBucket.objects.all():
#     print(obj)


In [20]:
import json
client_inspect = boto3.client('s3')
src_bucket = 'udacity-dend'
src_object = client_inspect.get_object(Bucket=src_bucket,Key='log_data/2018/11/2018-11-06-events.json')
df = pd.read_json(src_object['Body'], lines=True)

df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Adler,M,0,Barrera,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540836e+12,248,None,200,1541470364796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2""",100
1,Gustavo Cerati,Logged In,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Uno Entre 1000,200,1541470383796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2""",100
2,Limp Bizkit,Logged In,Adler,M,2,Barrera,270.49751,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Behind Blue Eyes,200,1541470632796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2""",100
3,None,Logged In,Samuel,M,0,Gonzalez,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540493e+12,252,None,200,1541473967796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4""",61
4,Mikel Erentxun,Logged In,Samuel,M,1,Gonzalez,178.83383,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,252,Frases Mudas,200,1541474048796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4""",61


In [29]:
import json
client_inspect = boto3.client('s3')
src_bucket = 'udacity-dend'
src_object = client_inspect.get_object(Bucket=src_bucket,Key='song-data/B/B/Y/TRBBYFJ128F42A22D4.json')
df = pd.read_json(src_object['Body'], lines=True)

df.head()

,song_id,num_songs,title,artist_name,artist_latitude,year,duration,artist_id,artist_longitude,artist_location
0,SOWVTAJ12A8C13C6D5,1,Jingle Bells,Jim Reeves,32.20049,1963,108.19873,ARZVTZN1187FB579D4,-94.35887,"Galloway, Panola County, TX"


## STEP 1: IAM ROLE
- Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [4]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)

1.1 Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dwhRole already exists.


In [5]:
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']


1.2 Attaching Policy


200

In [6]:
print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.3 Get the IAM role ARN
arn:aws:iam::697682206292:role/dwhRole


## STEP 2:  Redshift Cluster

- Create a RedShift Cluster
- For complete arguments to `create_cluster`, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster)

In [7]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

## 2.1 *Describe* the cluster to see its status
- run this block several times until the cluster status becomes `Available`

In [8]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', 1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.cxmu9btas4jt.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-6c790b14
7,NumberOfNodes,4


<h2> 2.2 Take note of the cluster <font color='red'> endpoint and role ARN </font> </h2>

<font color='red'>DO NOT RUN THIS unless the cluster status becomes "Available" </font>

In [10]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  dwhcluster.cxmu9btas4jt.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::697682206292:role/dwhRole


## STEP 3: Open an incoming  TCP port to access the cluster ednpoint

In [11]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-6db52338')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


## STEP 4: Make sure you can connect to the clusterConnect to the cluster

In [14]:
conn = redshift_connector.connect(
     host=DWH_ENDPOINT,
     database=DWH_DB,
     port=DWH_PORT,
     user=DWH_DB_USER,
     password=DWH_DB_PASSWORD
  )

## STEP 4: Sample Queries

In [15]:
cursor = conn.cursor()

In [16]:
cursor.execute("SELECT * FROM PG_TABLE_DEF;")

In [19]:
cursor.execute("SELECT COUNT(*) from public.staging_events")
result: tuple = cursor.fetchall()
print(result)

([8056],)


In [20]:
cursor.execute("SELECT COUNT(*) from public.staging_songs")
result: tuple = cursor.fetchall()
print(result)

([14896],)


In [21]:
cursor.execute("SELECT COUNT(*) from public.users")
result: tuple = cursor.fetchall()
print(result)

([104],)


In [22]:
cursor.execute("SELECT COUNT(*) from public.songs")
result: tuple = cursor.fetchall()
print(result)

([14896],)


In [23]:
cursor.execute("SELECT COUNT(*) from public.songplays")
result: tuple = cursor.fetchall()
print(result)

([1144],)


In [24]:
cursor.execute("SELECT COUNT(*) from public.time")
result: tuple = cursor.fetchall()
print(result)

([6813],)


In [25]:
cursor.execute("SELECT COUNT(*) from public.artists")
result: tuple = cursor.fetchall()
print(result)

([10025],)


In [26]:
query_most_played_song = """
    SELECT sp.song_id, 
        s.title, 
        count(*) AS count_song
    FROM songplays AS sp, songs AS s 
    WHERE s.song_id = sp.song_id  
    GROUP BY sp.song_id, s.title 
    ORDER BY count_song DESC
"""
cursor.execute(query_most_played_song)
result: tuple = cursor.fetchall()
print(result)

(['SOBONKR12A58A7A7E0', "You're The One", 37], ['SOTTKFE12A6D4F9AFE', 'Secrets', 17], ['SOZZHYJ12A8C140F50', 'Home', 13], ['SOBEUMD12AB018A9BC', 'Home', 13], ['SORKXUL12AB01821DA', 'Home', 13], ['SOWIIJD12A6D4FCAF0', 'Home', 13], ['SOUNZHU12A8AE47481', "I CAN'T GET STARTED", 9], ['SOHTKMO12AB01843B0', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 9], ['SOULTKQ12AB018A183', "Nothin' On You [feat. Bruno Mars] (Album Version)", 8], ['SOTSEFJ12AB0187FC3', 'Stronger', 7], ['SONENBB12A6D4FBE9F', 'Float On', 7], ['SOAEILM12A8C13AC54', 'Hello', 6], ['SOLZOBD12AB0185720', "Hey Daddy (Daddy's Home)", 6], ['SOZKYCJ12A6D4F480A', 'Thank You', 5], ['SOOUSHT12AC3DF485E', 'Thank You', 5], ['SOLFKYA12A8C13B082', 'Smile', 5], ['SOTNHIP12AB0183131', 'Make Her Say', 5], ['SOKLLZS12A8C13F208', 'Smile', 5], ['SOBHAQP12AB018B4D9', 'Just Dance', 5], ['SOJRCDU12AB0189D27', 'Smile', 5], ['SOARUPP12AB01842E0', 'Up Up & Away', 5], ['SOOCUWU12A67AD81AE', 'Smile', 5], ['SONQEYS12AF72AABC9', 'Mr. Jones', 4

In [28]:
query_list_played_song = """
    SELECT  sp.songplay_id,
        u.user_id,
        u.last_name,
        u.first_name,
        sp.start_time,
        a.name,
        s.title
    FROM songplays AS sp
            JOIN users   AS u ON (u.user_id = sp.user_id)
            JOIN songs   AS s ON (s.song_id = sp.song_id)
            JOIN artists AS a ON (a.artist_id = sp.artist_id)
            JOIN time    AS t ON (t.start_time = sp.start_time)
    LIMIT 200;
"""
cursor.execute(query_most_played_song)
result: tuple = cursor.fetchall()
print(result)

([583, 8, 'Summers', 'Kaylee', datetime.datetime(2018, 11, 1, 21, 11, 13), 'The Mars Volta', 'Eriatarka'], [567, 8, 'Summers', 'Kaylee', datetime.datetime(2018, 11, 7, 1, 42, 43), 'Yeah Yeah Yeahs', 'Heads Will Roll'], [319, 8, 'Summers', 'Kaylee', datetime.datetime(2018, 11, 27, 4, 25), 'Linkin Park', 'Given Up (Album Version)'], [921, 12, 'Rosales', 'Austin', datetime.datetime(2018, 11, 6, 8, 49, 19), 'Slim Dusty', 'Long Black Road'], [368, 12, 'Rosales', 'Austin', datetime.datetime(2018, 11, 12, 21, 4, 41), 'Frozen Plasma', 'Home'], [252, 12, 'Rosales', 'Austin', datetime.datetime(2018, 11, 12, 21, 4, 41), 'Working For A Nuclear Free City', 'Home'], [339, 12, 'Rosales', 'Austin', datetime.datetime(2018, 11, 12, 21, 4, 41), 'Gemma Hayes', 'Home'], [521, 12, 'Rosales', 'Austin', datetime.datetime(2018, 11, 12, 21, 4, 41), 'Eli Young Band', 'Home'], [164, 12, 'Rosales', 'Austin', datetime.datetime(2018, 11, 12, 21, 22, 1), 'Kid Cudi / Kanye West / Common', 'Up Up & Away'], [213, 12, 'R

In [ ]:
query_count_song_by_day = """
    SELECT t.day, 
        t.month, 
        t.year,
        count(*) AS count_song
    FROM songplays AS sp, time AS t
    WHERE sp.start_time = t.start_time
    GROUP BY t.day, t.month, t.year
    ORDER BY count_song DESC
"""
cursor.execute(query_count_song_by_day)
result: tuple = cursor.fetchall()
print(result)

## STEP 5: Clean up your resources

<b><font color='red'>DO NOT RUN THIS UNLESS YOU ARE SURE <br/> 
    We will be using these resources in the next exercises</span></b>

In [17]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.cxmu9btas4jt.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-6c790b14
7,NumberOfNodes,4


In [29]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'dwhuser',
  'DBName': 'dwh',
  'Endpoint': {'Address': 'dwhcluster.cxmu9btas4jt.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2023, 11, 8, 12, 31, 39, 284000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-6db52338',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-6c790b14',
  'AvailabilityZone': 'us-west-2a',
  'PreferredMaintenanceWindow': 'sat:13:30-sat:14:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible

- run this block several times until the cluster really deleted

In [87]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '200',
   'content-type': 'text/xml',
   'date': 'Sat, 16 Feb 2019 07:13:50 GMT',
   'x-amzn-requestid': '694f8d91-31ba-11e9-9438-d3ce9c613ef8'},
  'HTTPStatusCode': 200,
  'RequestId': '694f8d91-31ba-11e9-9438-d3ce9c613ef8',
  'RetryAttempts': 0}}